<a href="https://colab.research.google.com/github/minhaz1172/Deep-Learning/blob/main/transfer_learning(VGG16).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import dataset from kaggle**

# upload kaggle.json api in colab

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minhaz1172","key":"ae7abd8510b81a9fa7badffec1b23486"}'}

# **Move and set permissson**

In [2]:
import os
import zipfile

# Make directory and move kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set proper permissions
!chmod 600 ~/.kaggle/kaggle.json


# **install kaggle package**

In [3]:
!pip install -q kaggle


# **Find and download the dataset**

In [4]:

!kaggle datasets download -d salader/dogs-vs-cats


Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:08<00:00, 262MB/s]
100% 1.06G/1.06G [00:08<00:00, 130MB/s]


# unzip the dataset

In [5]:
# Unzip the dataset (replace the file name if needed)
!unzip -q dogs-vs-cats

# import necessary Liabraries

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

# **making a class of vgg16**
#  Load the VGG16 model with pretrained ImageNet weights.
# We exclude the top (classification) layers(output layers) since we'll be adding our own.

In [7]:
base_model=VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Create a sequential mode and add layers

In [8]:
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256,activation='relu')) # add a fully connected dense layer
model.add(Dropout(0.5)) # add dropout for regularization to help prevent overfitting.
model.add(Dense(1,activation='sigmoid')) # add a final output layer



# **Freeze the base model so that its weights do not get updated during the initial training phase**

In [9]:
base_model.trainable=False

# Define image dimension and batch size for our image generatio

In [10]:
img_height,img_width=224,224 ## VGG16 expects 224x224 images
batch_size=32  # number of images per gradient update

# Create data generation with augmentation

In [16]:
train_datagen=ImageDataGenerator(
    rescale=1./255, ## Normalize pixel values to [0,1]
    rotation_range=40,  # Randomly rotate images up to 40 degrees
    width_shift_range=0.2, # Randomly shift images horizontally by 20%
    height_shift_range=0.2, # Randomly shift images vertically by 20%
    zoom_range=0.2, #  # Randomly zoom in on images
     horizontal_flip=True, #  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in missing pixels after transformations

)


# create an imageDatagenerator for validation without augmentation(without rescaling

In [17]:
val_datagen=ImageDataGenerator(rescale=1./255)

# **Define the directory paths for training and validation data in your Colab folder**

In [18]:
train_dir='/content/train'
val_dir='/content/test'

# **define the directory path for training and validation data which is in my colab folder**

# Create a data generator that reads images from the training directory,
# applies augmentation, and resizes them to the required dimensions.

In [21]:
train_generator=train_datagen.flow_from_directory(
    train_dir, # directory with training images
    target_size=(img_height,img_width), # resize image with 224 by 224
    batch_size=batch_size, #number
    class_mode='categorical'
)


Found 20000 images belonging to 2 classes.


# create a data generator for validation data(no augmentation is applied)

In [22]:
validation_generator=val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5000 images belonging to 2 classes.
